In [30]:
# Import packages
import pyodbc
import json

In [31]:
# Load database configuration from JSON file
with open('config.json', 'r') as file:
    config = json.load(file)

# Set connection parameters
sql_server_name = config['sql_server_name']
sql_server_database_name = 'master' # Initialize as master, will be changed later on as we iterate through databases
sql_server_username = config['sql_server_username']
sql_server_password = config['sql_server_password']
sql_server_driver = config['sql_server_driver']

In [32]:
# Load SQL queries from files
with open('queries/get_databases.sql', 'r') as file:
    databases_query = file.read()

with open('queries/get_stored_procedures.sql', 'r') as file:
    stored_procedures_query = file.read()

with open('queries/get_tables_and_views.sql', 'r') as file:
    tables_views_query = file.read()

with open('queries/get_columns.sql', 'r') as file:
    columns_query = file.read()

with open('queries/get_stored_procedure_definitions.sql', 'r') as file:
    stored_procedure_definitions_query = file.read()

In [33]:
"""
DATABASE DATA EXTRACTION
for each database in the provided sql server extract the database name
"""

# Connect to master database
conn_master = pyodbc.connect(f'DRIVER={sql_server_driver};SERVER={sql_server_name};DATABASE={sql_server_database_name};UID={sql_server_username};PWD={sql_server_password}')
cursor_master = conn_master.cursor()

# Query to get the list of all databases
cursor_master.execute(databases_query)

# Fetch all query data
rows = cursor_master.fetchall()

# Initialize list to store database names excluding master
databases = []
for row in rows:
    if row.name.lower() != 'master':
        databases.append(row.name)

# Save database metadata as JSON format
database_metadata = {"databases": databases}

# Close the master database cursor and connection
if cursor_master:
    cursor_master.close()
if conn_master:
    conn_master.close()

In [34]:
# Initialize dictionaries to store metadata
stored_procedure_metadata = {}
table_metadata = {}

# Iterate over each database
for db in databases:
    # Connect to the database
    conn_db = pyodbc.connect(f'DRIVER={sql_server_driver};SERVER={sql_server_name};DATABASE={db};UID={sql_server_username};PWD={sql_server_password}')
    cursor_db = conn_db.cursor()

    """
    STORED PROCEDURE DATA EXTRACTION
    """

    # Query to get stored procedures
    cursor_db.execute(stored_procedures_query)
    objects = cursor_db.fetchall()
    stored_procedure_list = []

    for obj in objects:
        cursor_db.execute(stored_procedure_definitions_query, obj.ObjectID)
        definition = cursor_db.fetchone()
        
        # Ensure a definition is present
        if definition:
            definition_text = definition.definition
        else:
            definition_text = "No definition available"

        # Assign cursor values
        stored_procedure_list.append({
            "ObjectName": obj.ObjectName,
            "ObjectID": obj.ObjectID,
            "ObjectType": obj.ObjectType,
            "Definition": definition_text
        })

    # Write values to list
    stored_procedure_metadata[db] = stored_procedure_list

    """
    TABLE AND VIEW DATA EXTRACTION
    """

    # Initialize lists
    table_list = []
    column_list = []

    # Query to get user tables and views
    cursor_db.execute(tables_views_query)
    objects = cursor_db.fetchall()

    for obj in objects:
        
        # Query to get columns for each table/view
        cursor_db.execute(columns_query, obj.ObjectID)
        columns = cursor_db.fetchall()

        for col in columns:
            column_list.append({
                "ColumnName": col.ColumnName,
                "ColumnID": col.ColumnID,
                "DataType": col.DataType,
                "MaxLength": col.MaxLength,
                "IsNullable": col.IsNullable,
                "IsIdentity": col.IsIdentity
            })

        table_list.append({
            "ObjectName": obj.ObjectName,
            "ObjectID": obj.ObjectID,
            "ObjectType": obj.ObjectType,
            "Columns": column_list
        })

    table_metadata[db] = table_list

    # Close the database cursor and connection
    if cursor_db:
        cursor_db.close()
    if conn_db:
        conn_db.close()

In [35]:
# Write stored procedure metadata to JSON file
with open('temp/stored_procedure_metadata.json', 'w', encoding='utf-8') as jsonfile:
    json.dump(stored_procedure_metadata, jsonfile, ensure_ascii=False, indent=4)

# WRite table and view metadata to JSON file
with open('temp/table_and_view_metadata.json', 'w', encoding='utf-8') as jsonfile:
    json.dump(table_metadata, jsonfile, ensure_ascii=False, indent=4)

# Write database names to a JSON file
with open('temp/database_metadata.json', 'w', encoding='utf-8') as jsonfile:
    json.dump(database_metadata, jsonfile, ensure_ascii=False, indent=4)

## TO DO:

1. allow an input of table/view/stored procedure/database names to be excluded
2. account for database auto pause (delay/retry)
3. default to mssql but have an option to pass sql type which then select the queries